
## cn_stock/bars/daily

### Overwrite

In [1]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    with open("finance.duckdb.sql") as f:
        # conn.execute("drop table bars_daily_000001_SZ;")
        sql = f.read().format(tablename='000001_SZ', csv_path='datasources/000001.SZ')
        conn.execute(sql)
    conn.table('bars_daily_000001_SZ').show()

┌─────────────────────┬──────────┬───────┬───────┬───────┬───┬───────────────┬─────────┬─────────┬─────────┬─────────┐
│         dt          │   name   │ _open │ _high │ _low  │ … │ lb_down_count │  close  │  open   │  high   │   low   │
│     timestamp_s     │ varchar  │ float │ float │ float │   │     uint8     │  float  │  float  │  float  │  float  │
├─────────────────────┼──────────┼───────┼───────┼───────┼───┼───────────────┼─────────┼─────────┼─────────┼─────────┤
│ 1991-01-03 00:00:00 │ 深发展Ａ │  66.4 │  66.4 │  66.4 │ … │             0 │ 7.07077 │ 7.07077 │ 7.07077 │ 7.07077 │
│ 1991-01-04 00:00:00 │ 深发展Ａ │ 66.07 │ 66.07 │ 66.07 │ … │             0 │ 7.03562 │ 7.03562 │ 7.03562 │ 7.03562 │
│ 1991-01-07 00:00:00 │ 深发展Ａ │ 66.42 │ 66.42 │ 66.42 │ … │             0 │  7.0729 │  7.0729 │  7.0729 │  7.0729 │
│ 1991-01-09 00:00:00 │ 深发展Ａ │ 65.76 │ 65.76 │ 65.76 │ … │             0 │ 7.00261 │ 7.00261 │ 7.00261 │ 7.00261 │
│ 1991-01-10 00:00:00 │ 深发展Ａ │ 65.43 │ 65.43 │ 65.43 │ … │      

In [2]:
import duckdb

with duckdb.connect("datas/duckdb/finance_ticks_ctpfuture.db") as conn:
    with open("finance.duckdb.ticks.sql") as f:
        # conn.execute("drop table ticks_c;")
        sql = f.read().format(tablename='c', csv_path='datasources/c') # c 有3个G没有上传git了
        conn.execute(sql)
    conn.table('ticks_c').show()

┌──────────────────────┬────────────┬────────┬──────────┬───┬────────────┬────────────┬─────────────┬─────────────┐
│          dt          │ last_price │ volume │  amount  │ … │ bid_price1 │ ask_price1 │ bid_volume1 │ ask_volume1 │
│      timestamp       │   float    │ uint32 │  uint32  │   │   float    │   float    │   uint32    │   uint32    │
├──────────────────────┼────────────┼────────┼──────────┼───┼────────────┼────────────┼─────────────┼─────────────┤
│ 2017-01-04 09:00:0…  │     1521.0 │    985 │ 14988160 │ … │     1520.0 │     1521.0 │         143 │          15 │
│ 2017-01-04 09:00:0…  │     1521.0 │    384 │  5841550 │ … │     1520.0 │     1521.0 │         103 │          90 │
│ 2017-01-04 09:00:0…  │     1521.0 │     58 │   882110 │ … │     1520.0 │     1521.0 │         201 │          76 │
│ 2017-01-04 09:00:0…  │     1521.0 │     49 │   745150 │ … │     1520.0 │     1521.0 │         289 │          68 │
│ 2017-01-04 09:00:0…  │     1520.0 │    458 │  6963530 │ … │     1520.0

### Write

In [3]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    insert_sql = f"INSERT INTO bars_daily_000001_SZ SELECT dt,name,_open,_high,_low,_close,volume,amount,preclose,net_profit_ttm,cashflow_ttm,equity,asset,debt,debttoasset,net_profit_q,pe_ttm,pb,mkt_cap,mkt_cap_ashare,vip_buy_amt,vip_sell_amt,inst_buy_amt,inst_sell_amt,mid_buy_amt,mid_sell_amt,indi_buy_amt,indi_sell_amt,master_net_flow_in,master2_net_flow_in,vip_net_flow_in,mid_net_flow_in,inst_net_flow_in,indi_net_flow_in,total_sell_amt,total_buy_amt,net_flow_in,turnover,free_shares,total_shares,maxupordown,maxupordown_at_open,lb_up_count,lb_down_count,close,open,high,low FROM 'datasources/data_append.csv' where symbol='000001.SZ';"
    conn.execute(insert_sql)

### Delete

In [4]:
import duckdb

with duckdb.connect("datas/duckdb/test_finance.db") as conn:
    # 删除不存在的行不会报错
    insert_sql = f"DELETE FROM bars_daily_000001_SZ WHERE dt = '2024-09-02 00:00:00';"
    conn.execute(insert_sql)

### Read

In [6]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/test_finance.db"]):
    # df = qd.duckdb_get_array("test_finance", 'bars_daily_000001_SZ').df()
    # df = qd.duckdb_get_array("test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open']).df()
    # df = qd.duckdb_get_array("test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open'], filter="dt>'2024-08-29 00:00:00' and open > 10.08").df()
    # 最后一行(所有列)
    # df = qd.duckdb_get_array_last_rows("test_finance", 'bars_daily_000001_sz').df()
    # 最后一行(自定义列)
    # df = qd.duckdb_get_array_last_rows("test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open']).df()
    # 最后5行
    df = qd.duckdb_get_array_last_rows("test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open'], filter="dt<'2024-08-01 00:00:00'", N=5).df()
    # df = qd.duckdb_get_array_last_rows("test_finance", 'bars_daily_000001_SZ', N=1).df()
df

duckdb attach datas/duckdb/test_finance.db...
duckdb closing...
duckdb closed


,dt,name,open
0,2024-07-25,平安银行,10.12
1,2024-07-26,平安银行,10.10
2,2024-07-29,平安银行,10.02
3,2024-07-30,平安银行,10.10
4,2024-07-31,平安银行,10.08


In [8]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/test_finance.db"]):
    q = qd.duckdb_get_array("test_finance", 'bars_daily_000001_SZ', attrs=['dt', 'name', 'open'])
    recarray = qd.duckdb_fetchnumpy_to_recarray(q)
recarray

duckdb attach datas/duckdb/test_finance.db...
duckdb closing...
duckdb closed


rec.array([('1991-01-03T00:00:00', '深发展Ａ',  7.07077),
           ('1991-01-04T00:00:00', '深发展Ａ',  7.03562),
           ('1991-01-07T00:00:00', '深发展Ａ',  7.0729 ), ...,
           ('2024-08-28T00:00:00', '平安银行', 10.41   ),
           ('2024-08-29T00:00:00', '平安银行', 10.31   ),
           ('2024-08-30T00:00:00', '平安银行', 10.11   )],
          dtype=[('dt', '<M8[s]'), ('name', 'O'), ('open', '<f4')])

### Benchmark

以下为在大数据(全部5000+只股票的数据，一共5G)测试性能

In [10]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    %timeit qd.duckdb_get_array("finance", 'bars_daily_000001_SZ').fetchnumpy()
    %timeit  qd.duckdb_get_array("finance", 'bars_daily_000002_SZ', attrs=['dt', 'name', 'open']).fetchnumpy()
    %timeit  qd.duckdb_get_array("finance", 'bars_daily_000003_SZ', attrs=['dt', 'name', 'open'], filter="dt>'2024-08-29 00:00:00' and open > 10.08").fetchnumpy()
    # # 最后一行(所有列)
    %timeit  qd.duckdb_get_array_last_rows("finance", 'bars_daily_000005_SZ').fetchnumpy()
    # # 最后一行(自定义列)
    %timeit  qd.duckdb_get_array_last_rows("finance", 'bars_daily_000008_SZ', attrs=['dt', 'name', 'open']).fetchnumpy()
    # # 最后5行
    %timeit qd.duckdb_get_array_last_rows("finance", 'bars_daily_000010_SZ', attrs=['dt', 'name', 'open'], N=5).fetchnumpy()
    %timeit  qd.duckdb_get_array_last_rows("finance", 'bars_daily_000001_SZ', N=5).fetchnumpy()

duckdb attach datas/duckdb/finance.db...
5.32 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.15 ms ± 59.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
540 µs ± 18.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.61 ms ± 99 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.51 ms ± 57.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.35 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
7.93 ms ± 65 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
duckdb closing...
duckdb closed


以下为 duckdb_attach 函数测试，分为
- 本地文件
- 局域网内共享文件
- HTTP 文件

In [11]:
import quantdata as qd

def test_conn01():
    with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
        pass

test_conn01()

duckdb attach datas/duckdb/finance.db...
duckdb closing...
duckdb closed


In [12]:
import quantdata as qd

def test_conn02():
    with qd.open_duckdb(uris=["//DESKTOP-6NI30V4/bt_data/duckdb/finance.db"]):
        pass

test_conn02()

duckdb attach //DESKTOP-6NI30V4/bt_data/duckdb/finance.db...
duckdb closing...
duckdb closed


In [ ]:
import quantdata as qd

def test_conn03():
    with qd.open_duckdb(extensions=('httpfs',), uris=["http://192.168.2.40:9001/finance.db"]):
        pass

test_conn03()

以下为 fetchxxx() 函数测试

- 测试拉取一只股票所有数据. fetchnumpy 效率是最高的, 其次是 fetchdf, fetchall 最差

In [5]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    %timeit qd.duckdb_get_array(conn, "finance", 'bars_daily_000001_SZ').fetchnumpy()

6.33 ms ± 44.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    %timeit qd.duckdb_get_array("finance", 'bars_daily_000001_SZ').fetchdf()

10.2 ms ± 654 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    %timeit qd.duckdb_get_array("finance", 'bars_daily_000001_SZ').fetchall()

71.1 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


- 拉取少量数据（6个字段，5天的数据）fetchnumpy 和 fetchall 效率最优, 其次是 fetchdf

In [1]:
start_microsec = 1711929600000000 # datetime(2024,4,1)
till_microsec = 1712448000000000 # datetime(2024,4,7)

In [2]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    %timeit qd.get_data_ndarray("finance", 'bars_daily_000001_SZ', ["dt", "open", "high", "low", "close", "maxupordown"], start_microsec=start_microsec, till_microsec=till_microsec)

duckdb attach datas/duckdb/finance.db...
658 µs ± 9.76 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
duckdb closing...
duckdb closed


In [3]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    %timeit qd.get_data_df("finance", 'bars_daily_000001_SZ', ["dt", "open", "high", "low", "close", "maxupordown"], start_microsec=start_microsec, till_microsec=till_microsec)

duckdb attach datas/duckdb/finance.db...
1.18 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
duckdb closing...
duckdb closed


In [4]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    %timeit qd.get_data_recarray_s("finance", 'bars_daily_000001_SZ', ["dt", "open", "high", "low", "close", "maxupordown"], start_microsec=start_microsec, till_microsec=till_microsec)

duckdb attach datas/duckdb/finance.db...
768 µs ± 20 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
duckdb closing...
duckdb closed


对获取 recarray 函数`duckdb_fetchrecarray`进行测试：

- 底层用 fetchall. 大数据上效率最低

In [5]:
import quantdata as qd
import numpy as np

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    print("大数据:")
    %timeit qd.get_data_recarray_onlyfortest("finance", 'bars_daily_000001_SZ')
    print("小数据:")
    %timeit qd.get_data_recarray_onlyfortest("finance", 'bars_daily_000001_SZ', ["dt", "open", "high", "low", "close", "maxupordown"], start_microsec=start_microsec, till_microsec=till_microsec)

duckdb attach datas/duckdb/finance.db...
大数据:
62.4 ms ± 983 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
小数据:
765 µs ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
duckdb closing...
duckdb closed


- 底层用 fetchnumpy 效率均衡

In [6]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    print("大数据:")
    %timeit qd.get_data_recarray_s("finance", 'bars_daily_000001_SZ')
    print("小数据:")
    %timeit qd.get_data_recarray_s("finance", 'bars_daily_000001_SZ', ["dt", "open", "high", "low", "close", "maxupordown"], start_microsec=start_microsec, till_microsec=till_microsec)

duckdb attach datas/duckdb/finance.db...
大数据:
36.4 ms ± 2.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
小数据:
764 µs ± 17.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
duckdb closing...
duckdb closed


- 底层用 fetchdf. 大数据上效率最高

In [7]:
import quantdata as qd

with qd.open_duckdb(uris=["datas/duckdb/finance.db"]):
    print("大数据:")
    %timeit qd.get_data_recarray_l("finance", 'bars_daily_000001_SZ')
    print("小数据:")
    %timeit qd.get_data_recarray_l("finance", 'bars_daily_000001_SZ', ["dt", "open", "high", "low", "close", "maxupordown"], start_microsec=start_microsec, till_microsec=till_microsec)

duckdb attach datas/duckdb/finance.db...
大数据:
24.6 ms ± 387 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
小数据:
1.46 ms ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
duckdb closing...
duckdb closed
